<a href="https://colab.research.google.com/github/bhobho/udacity/blob/main/YT_PEFT_Finetune_Bloom7B_tagger.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Using 🤗 PEFT & bitsandbytes to finetune a LoRa checkpoint




In [ ]:
!pip install -q bitsandbytes datasets accelerate loralib
!pip install -q git+https://github.com/huggingface/transformers.git@main git+https://github.com/huggingface/peft.git

### Setup the model

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import torch
import torch.nn as nn
import bitsandbytes as bnb
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained(
    "bigscience/bloom-7b1",
    load_in_8bit=True,
    device_map='auto',
)

tokenizer = AutoTokenizer.from_pretrained("bigscience/bloom-7b1")

config.json: 100%|██████████| 739/739 [00:00<00:00, 3.90MB/s]
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
model.safetensors.index.json: 100%|██████████| 28.9k/28.9k [00:00<00:00, 52.3MB/s]
model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]
model-00001-of-00002.safetensors:   0%|          | 10.5M/9.98G [00:00<01:57, 84.5MB/s]
model-00001-of-00002.safetensors:   0%|          | 31.5M/9.98G [00:00<01:06, 148MB/s] 
model-00001-of-00002.safetensors:   1%|          | 62.9M/9.98G [00:00<01:09, 142MB/s]
model-00001-of-00002.safetensors:   1%|          | 83.9M/9.98G [00:00<01:05, 151MB/s]
model-00001-of-00002.safetensors:   1%|          | 115M/9.98G [00:00<01:06, 147MB/s] 
model-00001-of-00002.safetensors:   1%|▏         | 136M/9.98G [00:00<01:01, 161MB/s]
model-00001-of-00002.safetensors:   2%|▏         | 157M/9.98G [00:0

model-00001-of-00002.safetensors:  24%|██▍       | 2.38G/9.98G [00:12<00:44, 173MB/s]
model-00001-of-00002.safetensors:  24%|██▍       | 2.40G/9.98G [00:12<00:44, 171MB/s]
model-00001-of-00002.safetensors:  24%|██▍       | 2.43G/9.98G [00:12<00:40, 186MB/s]
model-00001-of-00002.safetensors:  25%|██▍       | 2.46G/9.98G [00:13<00:59, 127MB/s]
model-00001-of-00002.safetensors:  25%|██▍       | 2.49G/9.98G [00:13<00:58, 127MB/s]
model-00001-of-00002.safetensors:  25%|██▌       | 2.51G/9.98G [00:13<00:55, 135MB/s]
model-00001-of-00002.safetensors:  25%|██▌       | 2.53G/9.98G [00:13<00:52, 142MB/s]
model-00001-of-00002.safetensors:  26%|██▌       | 2.55G/9.98G [00:13<00:51, 144MB/s]
model-00001-of-00002.safetensors:  26%|██▌       | 2.57G/9.98G [00:13<00:50, 146MB/s]
model-00001-of-00002.safetensors:  26%|██▌       | 2.60G/9.98G [00:14<00:50, 147MB/s]
model-00001-of-00002.safetensors:  26%|██▋       | 2.62G/9.98G [00:14<00:48, 150MB/s]
model-00001-of-00002.safetensors:  26%|██▋       | 2.6

model-00001-of-00002.safetensors:  47%|████▋     | 4.69G/9.98G [00:25<00:25, 209MB/s]
model-00001-of-00002.safetensors:  47%|████▋     | 4.72G/9.98G [00:25<00:25, 209MB/s]
model-00001-of-00002.safetensors:  48%|████▊     | 4.75G/9.98G [00:26<00:24, 210MB/s]
model-00001-of-00002.safetensors:  48%|████▊     | 4.78G/9.98G [00:26<00:26, 193MB/s]
model-00001-of-00002.safetensors:  48%|████▊     | 4.80G/9.98G [00:26<00:29, 175MB/s]
model-00001-of-00002.safetensors:  48%|████▊     | 4.82G/9.98G [00:26<00:31, 163MB/s]
model-00001-of-00002.safetensors:  49%|████▊     | 4.84G/9.98G [00:26<00:29, 173MB/s]
model-00001-of-00002.safetensors:  49%|████▉     | 4.87G/9.98G [00:26<00:29, 170MB/s]
model-00001-of-00002.safetensors:  49%|████▉     | 4.89G/9.98G [00:26<00:30, 170MB/s]
model-00001-of-00002.safetensors:  49%|████▉     | 4.91G/9.98G [00:27<00:30, 168MB/s]
model-00001-of-00002.safetensors:  49%|████▉     | 4.93G/9.98G [00:27<00:30, 167MB/s]
model-00001-of-00002.safetensors:  50%|████▉     | 4.9

model-00001-of-00002.safetensors:  70%|██████▉   | 6.95G/9.98G [00:39<00:19, 153MB/s]
model-00001-of-00002.safetensors:  70%|██████▉   | 6.97G/9.98G [00:39<00:21, 141MB/s]
model-00001-of-00002.safetensors:  70%|███████   | 6.99G/9.98G [00:39<00:23, 125MB/s]
model-00001-of-00002.safetensors:  70%|███████   | 7.01G/9.98G [00:39<00:23, 127MB/s]
model-00001-of-00002.safetensors:  71%|███████   | 7.04G/9.98G [00:39<00:21, 137MB/s]
model-00001-of-00002.safetensors:  71%|███████   | 7.06G/9.98G [00:39<00:19, 151MB/s]
model-00001-of-00002.safetensors:  71%|███████   | 7.08G/9.98G [00:40<00:21, 135MB/s]
model-00001-of-00002.safetensors:  71%|███████   | 7.10G/9.98G [00:40<00:23, 121MB/s]
model-00001-of-00002.safetensors:  71%|███████▏  | 7.12G/9.98G [00:40<00:20, 137MB/s]
model-00001-of-00002.safetensors:  72%|███████▏  | 7.14G/9.98G [00:40<00:19, 144MB/s]
model-00001-of-00002.safetensors:  72%|███████▏  | 7.16G/9.98G [00:40<00:17, 157MB/s]
model-00001-of-00002.safetensors:  72%|███████▏  | 7.1

model-00001-of-00002.safetensors:  89%|████████▉ | 8.88G/9.98G [00:50<00:05, 201MB/s]
model-00001-of-00002.safetensors:  89%|████████▉ | 8.90G/9.98G [00:50<00:05, 202MB/s]
model-00001-of-00002.safetensors:  89%|████████▉ | 8.92G/9.98G [00:50<00:05, 202MB/s]
model-00001-of-00002.safetensors:  90%|████████▉ | 8.94G/9.98G [00:50<00:05, 202MB/s]
model-00001-of-00002.safetensors:  90%|████████▉ | 8.97G/9.98G [00:50<00:05, 201MB/s]
model-00001-of-00002.safetensors:  90%|█████████ | 8.99G/9.98G [00:50<00:04, 200MB/s]
model-00001-of-00002.safetensors:  90%|█████████ | 9.01G/9.98G [00:50<00:04, 200MB/s]
model-00001-of-00002.safetensors:  90%|█████████ | 9.03G/9.98G [00:50<00:04, 199MB/s]
model-00001-of-00002.safetensors:  91%|█████████ | 9.05G/9.98G [00:50<00:04, 194MB/s]
model-00001-of-00002.safetensors:  91%|█████████ | 9.07G/9.98G [00:51<00:05, 180MB/s]
model-00001-of-00002.safetensors:  91%|█████████ | 9.09G/9.98G [00:51<00:05, 154MB/s]
model-00001-of-00002.safetensors:  91%|█████████▏| 9.1

model-00002-of-00002.safetensors:  20%|█▉        | 818M/4.16G [00:04<00:16, 206MB/s]
model-00002-of-00002.safetensors:  20%|██        | 839M/4.16G [00:05<00:16, 201MB/s]
model-00002-of-00002.safetensors:  21%|██        | 860M/4.16G [00:05<00:16, 202MB/s]
model-00002-of-00002.safetensors:  21%|██        | 881M/4.16G [00:05<00:16, 200MB/s]
model-00002-of-00002.safetensors:  22%|██▏       | 902M/4.16G [00:05<00:16, 201MB/s]
model-00002-of-00002.safetensors:  22%|██▏       | 933M/4.16G [00:05<00:15, 204MB/s]
model-00002-of-00002.safetensors:  23%|██▎       | 954M/4.16G [00:05<00:15, 205MB/s]
model-00002-of-00002.safetensors:  23%|██▎       | 975M/4.16G [00:05<00:15, 204MB/s]
model-00002-of-00002.safetensors:  24%|██▍       | 996M/4.16G [00:05<00:15, 202MB/s]
model-00002-of-00002.safetensors:  25%|██▍       | 1.03G/4.16G [00:06<00:15, 205MB/s]
model-00002-of-00002.safetensors:  25%|██▌       | 1.05G/4.16G [00:06<00:15, 203MB/s]
model-00002-of-00002.safetensors:  26%|██▌       | 1.07G/4.16G 

model-00002-of-00002.safetensors:  68%|██████▊   | 2.82G/4.16G [00:16<00:07, 172MB/s]
model-00002-of-00002.safetensors:  68%|██████▊   | 2.84G/4.16G [00:16<00:07, 171MB/s]
model-00002-of-00002.safetensors:  69%|██████▉   | 2.86G/4.16G [00:16<00:07, 168MB/s]
model-00002-of-00002.safetensors:  69%|██████▉   | 2.88G/4.16G [00:17<00:07, 163MB/s]
model-00002-of-00002.safetensors:  70%|██████▉   | 2.90G/4.16G [00:17<00:08, 153MB/s]
model-00002-of-00002.safetensors:  70%|███████   | 2.93G/4.16G [00:17<00:07, 155MB/s]
model-00002-of-00002.safetensors:  71%|███████   | 2.95G/4.16G [00:17<00:07, 157MB/s]
model-00002-of-00002.safetensors:  71%|███████▏  | 2.97G/4.16G [00:17<00:07, 153MB/s]
model-00002-of-00002.safetensors:  72%|███████▏  | 2.99G/4.16G [00:17<00:08, 143MB/s]
model-00002-of-00002.safetensors:  72%|███████▏  | 3.01G/4.16G [00:17<00:08, 139MB/s]
model-00002-of-00002.safetensors:  73%|███████▎  | 3.03G/4.16G [00:18<00:08, 140MB/s]
model-00002-of-00002.safetensors:  73%|███████▎  | 3.0

### Freezing the original weights


In [ ]:
for param in model.parameters():
  param.requires_grad = False  # freeze the model - train adapters later
  if param.ndim == 1:
    # cast the small parameters (e.g. layernorm) to fp32 for stability
    param.data = param.data.to(torch.float32)

model.gradient_checkpointing_enable()  # reduce number of stored activations
model.enable_input_require_grads()

class CastOutputToFloat(nn.Sequential):
  def forward(self, x): return super().forward(x).to(torch.float32)
model.lm_head = CastOutputToFloat(model.lm_head)

### Setting up the LoRa Adapters

In [ ]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [ ]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=16, #attention heads
    lora_alpha=32, #alpha scaling
    # target_modules=["q_proj", "v_proj"], #if you know the
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM" # set this for CLM or Seq2Seq
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 7864320 || all params: 7076880384 || trainable%: 0.11112693126452029


## Data

In [ ]:
import transformers
from datasets import load_dataset
data = load_dataset("Abirate/english_quotes")


Extracting data files: 100%|██████████| 1/1 [00:00<00:00, 803.97it/s]
Generating train split: 2508 examples [00:00, 111734.06 examples/s]


In [ ]:
def merge_columns(example):
    example["prediction"] = example["quote"] + " ->: " + str(example["tags"])
    return example

data['train'] = data['train'].map(merge_columns)
data['train']["prediction"][:5]

Map: 100%|██████████| 2508/2508 [00:00<00:00, 10163.35 examples/s]


["“Be yourself; everyone else is already taken.” ->: ['be-yourself', 'gilbert-perreira', 'honesty', 'inspirational', 'misattributed-oscar-wilde', 'quote-investigator']",
 "“I'm selfish, impatient and a little insecure. I make mistakes, I am out of control and at times hard to handle. But if you can't handle me at my worst, then you sure as hell don't deserve me at my best.” ->: ['best', 'life', 'love', 'mistakes', 'out-of-control', 'truth', 'worst']",
 "“Two things are infinite: the universe and human stupidity; and I'm not sure about the universe.” ->: ['human-nature', 'humor', 'infinity', 'philosophy', 'science', 'stupidity', 'universe']",
 "“So many books, so little time.” ->: ['books', 'humor']",
 "“A room without books is like a body without a soul.” ->: ['books', 'simile', 'soul']"]

In [ ]:
data['train'][0]

{'quote': '“Be yourself; everyone else is already taken.”',
 'author': 'Oscar Wilde',
 'tags': ['be-yourself',
  'gilbert-perreira',
  'honesty',
  'inspirational',
  'misattributed-oscar-wilde',
  'quote-investigator'],
 'prediction': "“Be yourself; everyone else is already taken.” ->: ['be-yourself', 'gilbert-perreira', 'honesty', 'inspirational', 'misattributed-oscar-wilde', 'quote-investigator']"}

In [ ]:
data = data.map(lambda samples: tokenizer(samples['prediction']), batched=True)

Map: 100%|██████████| 2508/2508 [00:00<00:00, 4507.11 examples/s]


In [ ]:
data

DatasetDict({
    train: Dataset({
        features: ['quote', 'author', 'tags', 'prediction', 'input_ids', 'attention_mask'],
        num_rows: 2508
    })
})

### Training

In [ ]:

trainer = transformers.Trainer(
    model=model,
    train_dataset=data['train'],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=4,
        gradient_accumulation_steps=4,
        warmup_steps=100,
        max_steps=200,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=1,
        output_dir='outputs'
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

/opt/conda/lib/python3.10/site-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Step,Training Loss
1,3.084300
2,3.196800
3,3.135800
4,3.374700
5,3.399100
6,3.498200
7,2.961300
8,3.150000
9,3.453600
10,3.120700


OutOfMemoryError: CUDA out of memory. Tried to allocate 3.40 GiB (GPU 0; 14.76 GiB total capacity; 11.31 GiB already allocated; 1.06 GiB free; 12.95 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

## Share adapters on the 🤗 Hub

In [ ]:
model.push_to_hub("samwit/bloom-7b1-lora-tagger",
                  use_auth_token=True,
                  commit_message="basic training",
                  private=True)

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

adapter_model.bin:   0%|          | 0.00/31.5M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/samwit/bloom-7b1-lora-tagger/commit/62cfae6c87a7d657b2bd3e6e2abac2d5a7d07caf', commit_message='basic training', commit_description='', oid='62cfae6c87a7d657b2bd3e6e2abac2d5a7d07caf', pr_url=None, pr_revision=None, pr_num=None)

## Load adapters from the Hub

In [ ]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer

peft_model_id = "samwit/bloom-7b1-lora-tagger"
config = PeftConfig.from_pretrained(peft_model_id)
model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path, return_dict=True, load_in_8bit=True, device_map='auto')
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

# Load the Lora model
model = PeftModel.from_pretrained(model, peft_model_id)


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching /usr/local/cuda/lib64...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 8.0
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.9/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: /usr/lib64-nvidia did not contain libcudart.so as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('--listen_host=172.28.0.12 --target_host=172.28.0.12 --tunnel_background_save_url=https'), PosixPath('//colab.research.google.com/tun/m/cc48301118ce562b961b3c22d803539adc1e0c19/gpu-a100-s-c1xr75l0apxr --tunnel_background_save_delay=10s --tunnel_periodic_background_save_frequency=30m0s --enable_output_coalescing=true --output_coalescing_required

Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

## Inference

In [ ]:
batch = tokenizer("“Training models with PEFT and LoRa is cool” ->: ", return_tensors='pt')

with torch.cuda.amp.autocast():
  output_tokens = model.generate(**batch, max_new_tokens=50)

print('\n\n', tokenizer.decode(output_tokens[0], skip_special_tokens=True))



 “Training models with PEFT and LoRa is cool” ->:  ['training', 'teaching']

A:

I think the best way to describe it is to say that it is a combination of the two. The idea is that you can train a model on a dataset, and then you can use the
